In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [ ]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(8, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(16, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(20, activation='relu'))

In [ ]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0

#### 組裝

In [ ]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0899 - accuracy: 0.0951
Epoch 2/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0898 - accuracy: 0.1891
Epoch 3/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0896 - accuracy: 0.2665
Epoch 4/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0894 - accuracy: 0.2768
Epoch 5/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0891 - accuracy: 0.2855
Epoch 6/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0886 - accuracy: 0.3040
Epoch 7/12
469/469 [==============================] - 25s 54ms/step - loss: 0.0874 - accuracy: 0.3407
Epoch 8/12
469/469 [==============================] - 26s 54ms/step - loss: 0.0841 - accuracy: 0.3839
Epoch 9/12
469/469 [==============================] - 26s 55ms/step - loss: 0.0700 - accuracy: 0.4895
Epoch 10/12
469/469 [==============================] - 25s 54ms/step - loss: 0.043

### Step 3. 預測

In [ ]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [ ]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0223 - accuracy: 0.8474


In [ ]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 84.74%


In [ ]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [ ]:
from ipywidgets import interact_manual

In [ ]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.h5')